In [16]:
import tensorflow as tf
import numpy as np

import os
np.random.seed(0)

### 1. Setup the Wavefunction architecture

In [26]:
class RNNwavefunction(object):
    def __init__(self,systemsize,cell=tf.compat.v1.nn.rnn_cell.GRUCell,activation=tf.nn.relu,units=[15]):
        """ 
            RNN wave function architecture
            ------------------------------------------------------------------------
            Parameters:
            
            systemsize:  int, number of sites
            cell:        a tensorflow RNN 
            units:       list of int, each entry defines the number of hidden units for each RNN building block        
        """
        self.graph       = tf.Graph()
        self.scope       = 'RNNwavefunction' #Label of our wavefunction
        self.N           = systemsize
        
        with self.graph.as_default(): #everything inside will belong to self.graph
            with tf.compat.v1.variable_scope(self.scope): #make sure that this will be asigned to "RNNwavefunction" 
                tf.compat.v1.set_random_seed(1234)
                self.rnn = tf.compat.v1.nn.rnn_cell.MultiRNNCell([cell(units[n],activation=activation,name='GRU_{0}'.format(n)) for n in range(len(units))])
                print(self.rnn)
                # define the dense layer
                self.dense = tf.compat.v1.layers.Dense(2,activation=tf.nn.softmax,name='wf_dense')
                
    def sample(self,numsamples,inputdim):
        """
            generate samples from a probability distribution parametrized by a recurrent network
            ------------------------------------------------------------------------
            Parameters: 
            
            numsamples:      int, number of samples to be produced
            inputdim:        int, hilbert space dimension
    
            ------------------------------------------------------------------------
            Returns:         a tuple (samples,log-probs)
             
            samples:         tf.Tensor of shape (numsamples,systemsize), the samples in integer encoding 
        """ 
        self.inputdim   = inputdim
        self.numsamples = numsamples
        self.outputdim  = inputdim
        
        with self.graph.as_default():
            with tf.compat.v1.variable_scope(self.scope):
                samples = []
                one_hot_samples = []
                probs = []
                # define inputs for the RNN 
                inputs     = np.zeros((self.numsamples,self.inputdim))
                inputs     = tf.constant(dtype=tf.float32,value=inputs,shape=[self.numsamples,self.inputdim])
                # initialize the states tensors with zero (first hidden states)
                rnn_state  = self.rnn.zero_state(self.numsamples,dtype=tf.float32)
                
                for n in range(0,self.N):
                    #compute the next hidden state of the RNN
                    rnn_output, rnn_state = self.rnn(inputs, rnn_state)
                    print(rnn_output)
                    #apply the Softmax layer
                    output = self.dense(rnn_output)
                    probs.append(output)
                    #sample from the probability
                    temp   = tf.reshape(tf.random.categorical(tf.compat.v1.log(output),num_samples=1),[-1,]) 
                    samples.append(temp)
                    inputs = tf.one_hot(temp,depth=self.outputdim)
                    one_hot_samples.append(inputs)

            self.samples    = tf.stack(values=samples,axis=1)
            temp            = tf.transpose(tf.stack(values=probs,axis=2),perm=[0,2,1])
            one_hot_samples = tf.transpose(tf.stack(values=one_hot_samples,axis=2),perm=[0,2,1])
            self.log_probs  = tf.reduce_sum(tf.compat.v1.log(tf.reduce_sum(tf.multiply(temp,one_hot_samples),axis=2)),axis=1)            
            print(self.log_probs)
            return self.samples, self.log_probs
        
        
    
    def log_probability(self,samples,inputdim):
        """
            calculate the log-probabilities of samples
            ------------------------------------------------------------------------
            Parameters: 
            
            samples:         tf.Tensor, a tf.placeholder of shape (number of samples,system-size) 
                             containing the input samples in integer encoding
            inputdim:        int, dimension of the input space
                      
            ------------------------------------------------------------------------         
            Returns: 
            log-probs        tf.Tensor of shape (number of samples,), the log-probability of each sample         
            """
        
        self.inputdim   = inputdim
        self.outputim   = inputdim
        self.numsamples = numsamples
        
        with self.graph.as_default():

            with tf.compat.v1.variable_scope(self.scope):         
                probs      = []
                
                # define inputs for the RNN 
                inputs     = np.zeros((self.numsamples,self.inputdim)).astype(np.float32)
                inputs     = tf.constant(dtype=tf.float32,value=inputs,shape=[self.numsamples,self.inputdim])
                # initialize the states tensors with zero (first hidden states)
                rnn_state  = self.rnn.zero_state(self.numsamples,dtype=tf.float32)
                
                for n in range(0,self.N):
                    #compute the next hidden state of the RNN
                    rnn_output, rnn_state = self.rnn(inputs, rnn_state)
                    #apply the Softmax layer
                    output = self.dense(rnn_output)
                    #sample from the probability
                    probs.append(output)
                    inputs = tf.reshape(tf.one_hot(tf.reshape(tf.slice(samples,begin=[np.int32(0),np.int32(n)],size=[np.int32(-1),np.int32(1)]),shape=[self.numsamples]),depth=self.outputdim),shape=[self.numsamples,self.inputdim])
        

            temp            = tf.transpose(tf.stack(values=probs,axis=2),perm=[0,2,1])
            one_hot_samples = tf.one_hot(samples,depth=self.inputdim)
            self.log_probs  = tf.reduce_sum(tf.compat.v1.log(tf.reduce_sum(tf.multiply(temp,one_hot_samples),axis=2)),axis=1)            
            
            return self.log_probs


#### Initialization and first tests

In [27]:
units=[15] #list containing the number of hidden units for each layer of the networks

N          = 10
input_dim  = 2
numsamples = 5 #only for initialization; later I'll use a much larger value (see below)


wf         = RNNwavefunction(N,units=units) #contains the graph with the RNNs
samples    = wf.sample(numsamples,input_dim) #call this function once to create the dense layers
print(samples)
with wf.graph.as_default(): #now initialize everything 
    #placeholder for inputs and learning rate
    inputs       = tf.compat.v1.placeholder(dtype=tf.int32,shape=[numsamples,N]) 
    learningrate = tf.compat.v1.placeholder(dtype=tf.float32,shape=[])
    probs        = wf.log_probability(inputs,input_dim)
    optimizer    = tf.compat.v1.train.AdamOptimizer(learning_rate=learningrate)
    init         = tf.compat.v1.global_variables_initializer()

Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_1:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_2:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_3:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_4:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_5:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_6:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_7:0", shape=(5, 15), dtype=float32)
Tensor("RNNwavefunction_1/RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/add_8:0", shape=(5, 15), dtype=float32)
Tensor("RNNw

In [28]:
#Start the session
sess = tf.compat.v1.Session(graph=wf.graph)
sess.run(init)
generated_samples, generated_probs = sess.run(samples)  
print(generated_samples)
print(generated_probs)

[[0 0 1 1 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 1 1]
 [0 1 1 0 1 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 1 0 0 1 1 0 0 1 1]]
[-7.2338486 -6.595579  -6.75748   -7.067438  -7.1174088]


#### Parameters of the RNN wave function:

In [30]:
# count the number of parameters
with wf.graph.as_default():
    variables_names = [v.name for v in tf.compat.v1.trainable_variables()]
    sum = 0
    for v in variables_names:
        print(v)
        value = sess.run(v)
        sum += len(value)
    print('The number of variational parameters of the pRNN wavefunction is {0}'.format(sum))
    print('\n')

RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/gates/kernel:0
RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/gates/bias:0
RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/candidate/kernel:0
RNNwavefunction/multi_rnn_cell/cell_0/GRU_0/candidate/bias:0
RNNwavefunction/wf_dense/kernel:0
RNNwavefunction/wf_dense/bias:0
The number of variational parameters of the pRNN wavefunction is 870




### 2. Calculate the matrix elements (here 1D TFIM)

$$ E_{\theta}^{loc}(x) = \frac{<x|H|\psi_\theta>}{<x|\psi_\theta>} = H_{diag}(x)+H_{offd}(x)\frac{<x^{\prime}|\psi_\theta>}{<x|\psi_\theta>} $$
with $\hat{H}_{offd}|x^{\prime}>=H_{offd}(x)|x^{\prime}>$ and ${<x|\psi_\theta>}$ given by the square root of the exponential of rnn.log_probability(x) defined above.

In [107]:
def XXZ1D_MatrixElements(Jp, Jz, samples):
    """ 
    Calculate the local energies of 1D XXZ model given a set of set of samples.
    Returns: The local energies that correspond to the input samples.
    Inputs:
    - sample: (num_samples, N)
    - Jp: float
    - Jz: float
    """
    
    N = samples.shape[1]
    
    #diagonal elements
    diag_matrixelements = np.zeros((numsamples), dtype = np.float64)
    for i in range(N): #diagonal elements from the SzSz term
        values = samples[:,i]+samples[:,(i+1)%N]
        valuesT = np.copy(values)
        valuesT[values==2] = +1 #If both spins are up
        valuesT[values==0] = +1 #If both spins are down
        valuesT[values==1] = -1 #If they are opposite

        diag_matrixelements += valuesT*Jz*0.25
    
    #off-diagonal elements
    offd_matrixelements = np.zeros((numsamples), dtype = np.float64)
    for i in range(N): 
        values = samples[:,i]+samples[:,(i+1)%N]
        valuesT = np.copy(values)
        #flip the spins
        new_samples = samples.copy()
        new_samples[:,i+1] = samples[:,i]
        new_samples[:,i]   = samples[:,i+1]
        valuesT[values==2] = 0 #If both spins are up
        valuesT[values==0] = 0 #If both spins are down
        valuesT[values==1] = 1 #If they are opposite

        offd_matrixelements += valuesT*Jp*0.5
    return matrixelements

def XXZ1D_Eloc(Jp, Jz, samples, RNN):
    """ 
    Calculate the local energies of 1D XXZ model given a set of set of samples.
    Returns: The local energies that correspond to the input samples.
    Inputs:
    - sample: (num_samples, N)
    - Jp: float
    - Jz: float
    """
    N          = samples.shape[1]
    numsamples = samples.shape[0]
    with RNN.graph.as_default():
        samples_placeholder = tf.compat.v1.placeholder(dtype=tf.int32,shape=(None,N))
        log_probs_tensor    = wf.log_probability(samples_placeholder,inputdim=2)

    queue_samples       = np.zeros((N+1, numsamples, N), dtype = np.int32) 
    log_probs           = np.zeros((N+1)*numsamples, dtype=np.float64) 
    
    #diagonal elements
    diag_Eloc = np.zeros((numsamples), dtype = np.float64)
    for i in range(N): #diagonal elements from the SzSz term
        values             = samples[:,i]+samples[:,(i+1)%N]
        valuesT            = np.copy(values)
        valuesT[values==2] = +1 #If both spins are up
        valuesT[values==0] = +1 #If both spins are down
        valuesT[values==1] = -1 #If they are opposite

        diag_matrixelements = valuesT*Jz*0.25
        Eloc = diag_matrixelements
    queue_samples[0] = samples
    
    #off-diagonal elements
    offd_Eloc = np.zeros((numsamples), dtype = np.float64)
    for i in range(N): 
        values = samples[:,i]+samples[:,(i+1)%N]
        valuesT = np.copy(values)
        #flip the spins
        new_samples = samples.copy()
        new_samples[:,(i+1)%N] = samples[:,i]
        new_samples[:,i]   = samples[:,(i+1)%N]
        valuesT[values==2] = 0 #If both spins are up
        valuesT[values==0] = 0 #If both spins are down
        valuesT[values==1] = 1 #If they are opposite

        offd_matrixelements = valuesT*Jp*0.5
        queue_samples[i+1] = new_samples
    queue_samples_reshaped = np.reshape(queue_samples, [(N+1)*numsamples, N])
    log_probs = sess.run(log_probs_tensor)
    log_probs_reshaped = np.reshape(log_probs, [N+1,numsamples])
    Eloc += offd_matrixelements.dot(np.exp(0.5*log_probs_reshaped[1:,:]-0.5*log_probs_reshaped[0,:]))
    
    return Eloc
    
    

In [108]:
XXZ1D_Eloc(1, 0, generated_samples, wf)

TypeError: Fetch argument array([[0, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 0, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 1, 0],
       [1, 0, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int32) has invalid type <class 'numpy.ndarray'>, must be a string or Tensor. (Can not convert a ndarray into a Tensor or Operation.)